In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
#Initilize Sparksession
spark = SparkSession.builder.appName("WordCount").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/22 08:40:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/22 08:40:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
sc=spark.sparkContext

In [7]:
customerdf=spark.read.option("inferSchema", True).option("header", True).option("sep",'\t').csv("/home/labuser/Documents/Retail_Dataset/customer.csv")
lineitemdf=spark.read.option("inferSchema", True).option("header", True).option("sep",'\t').csv("/home/labuser/Documents/Retail_Dataset/lineitem.csv")
nationdf=spark.read.option("inferSchema", True).option("header", True).option("sep",'\t').csv("/home/labuser/Documents/Retail_Dataset/nation.csv")
ordersdf=spark.read.option("inferSchema", True).option("header", True).option("sep",'\t').csv("/home/labuser/Documents/Retail_Dataset/orders.csv")
partdf=spark.read.option("inferSchema", True).option("header", True).option("sep",'\t').csv("/home/labuser/Documents/Retail_Dataset/part.csv")
partsuppdf=spark.read.option("inferSchema", True).option("header", True).option("sep",'\t').csv("/home/labuser/Documents/Retail_Dataset/partsupp.csv")
regiondf=spark.read.option("inferSchema", True).option("header", True).option("sep",'\t').csv("/home/labuser/Documents/Retail_Dataset/region.csv")
supplierdf=spark.read.option("inferSchema", True).option("header", True).option("sep",'\t').csv("/home/labuser/Documents/Retail_Dataset/supplier.csv")

In [8]:
customerdf.printSchema()

root
 |-- C_CUSTKEY: integer (nullable = true)
 |-- C_NAME: string (nullable = true)
 |-- C_ADDRESS: string (nullable = true)
 |-- C_NATIONKEY: integer (nullable = true)
 |-- C_PHONE: string (nullable = true)
 |-- C_ACCTBAL: double (nullable = true)
 |-- C_MKTSEGMENT: string (nullable = true)
 |-- C_COMMENT: string (nullable = true)



In [9]:
lineitemdf.printSchema()

root
 |-- L_ORDERKEY: integer (nullable = true)
 |-- L_PARTKEY: integer (nullable = true)
 |-- L_SUPPKEY: integer (nullable = true)
 |-- L_LINENUMBER: integer (nullable = true)
 |-- L_QUANTITY: double (nullable = true)
 |-- L_EXTENDEDPRICE: double (nullable = true)
 |-- L_DISCOUNT: double (nullable = true)
 |-- L_TAX: double (nullable = true)
 |-- L_RETURNFLAG: string (nullable = true)
 |-- L_LINESTATUS: string (nullable = true)
 |-- L_SHIPDATE: date (nullable = true)
 |-- L_COMMITDATE: date (nullable = true)
 |-- L_RECEIPTDATE: date (nullable = true)
 |-- L_SHIPINSTRUCT: string (nullable = true)
 |-- L_SHIPMODE: string (nullable = true)
 |-- L_COMMENT: string (nullable = true)



In [10]:
nationdf.printSchema()

root
 |-- N_NATIONKEY: integer (nullable = true)
 |-- N_NAME: string (nullable = true)
 |-- N_REGIONKEY: integer (nullable = true)
 |-- N_COMMENT: string (nullable = true)



In [11]:
ordersdf.printSchema()

root
 |-- O_ORDERKEY: integer (nullable = true)
 |-- O_CUSTKEY: integer (nullable = true)
 |-- O_ORDERSTATUS: string (nullable = true)
 |-- O_TOTALPRICE: double (nullable = true)
 |-- O_ORDERDATE: date (nullable = true)
 |-- O_ORDERPRIORITY: string (nullable = true)
 |-- O_CLERK: string (nullable = true)
 |-- O_SHIPPRIORITY: integer (nullable = true)
 |-- O_COMMENT: string (nullable = true)



In [36]:
joindf = customerdf.join(ordersdf, customerdf.C_CUSTKEY == ordersdf.O_CUSTKEY, how="inner"). \
join(lineitemdf, ordersdf.O_ORDERKEY == lineitemdf.L_ORDERKEY, how="inner"). \
select(customerdf["*"],ordersdf["O_ORDERKEY"], ordersdf["O_ORDERSTATUS"], ordersdf["O_TOTALPRICE"], ordersdf["O_ORDERDATE"], \
       lineitemdf["L_LINENUMBER"],lineitemdf["L_QUANTITY"])


In [37]:
joindf.show(2)

+---------+------------------+----------+-----------+---------------+---------+------------+--------------------+----------+-------------+------------+-----------+------------+----------+
|C_CUSTKEY|            C_NAME| C_ADDRESS|C_NATIONKEY|        C_PHONE|C_ACCTBAL|C_MKTSEGMENT|           C_COMMENT|O_ORDERKEY|O_ORDERSTATUS|O_TOTALPRICE|O_ORDERDATE|L_LINENUMBER|L_QUANTITY|
+---------+------------------+----------+-----------+---------------+---------+------------+--------------------+----------+-------------+------------+-----------+------------+----------+
|      370|Customer#000000370|oyAPndV IN|         12|22-524-280-8721|  8982.79|   FURNITURE|ges. final packag...|         1|            O|   172799.49| 1996-01-02|           6|      32.0|
|      370|Customer#000000370|oyAPndV IN|         12|22-524-280-8721|  8982.79|   FURNITURE|ges. final packag...|         1|            O|   172799.49| 1996-01-02|           5|      24.0|
+---------+------------------+----------+-----------+-------

In [14]:
grpdf = ordersdf.groupBy("O_CLERK").sum("O_TOTALPRICE")

In [15]:
grpdf.show()

+---------------+------------------+
|        O_CLERK| sum(O_TOTALPRICE)|
+---------------+------------------+
|Clerk#000000024|        2078084.41|
|Clerk#000000288|        1431028.03|
|Clerk#000000236|        2571896.97|
|Clerk#000000093|3388791.1500000004|
|Clerk#000000124|        1683034.58|
|Clerk#000000011|2032237.1699999995|
|Clerk#000000114|1596777.3599999999|
|Clerk#000000022|         1250757.7|
|Clerk#000000474|1669808.3399999999|
|Clerk#000000723| 923212.7899999999|
|Clerk#000000903|2984974.0700000008|
|Clerk#000000398|        1799587.31|
|Clerk#000000744|         2078169.2|
|Clerk#000000912|1728371.5500000005|
|Clerk#000000644|         2357482.5|
|Clerk#000000766|        1941942.28|
|Clerk#000000674|         941649.11|
|Clerk#000000821|2854880.0399999996|
|Clerk#000000424|1509396.7800000003|
|Clerk#000000087|        1396516.85|
+---------------+------------------+
only showing top 20 rows



In [38]:
test = joindf.repartition(8)

In [39]:
test.rdd.getNumPartitions()

8

In [40]:
test.coalesce(1).write.mode("append").parquet("/home/labuser/Documents/output")